In [1]:
import os
import sys
import json
if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import sumolib
import importlib
import LLAMAconnect
import subprocess
import shutil
from pprint import pprint
import random
import vehParameters
import traci

In [2]:
net = sumolib.net.readNet('osm.net.xml')
parkingAreas = list(sumolib.output.parse('osm_stops.add.xml', "parkingArea"))

In [3]:
## Script for generating alternative routes
duaiterate_path = "/usr/share/sumo/tools/assign/duaIterate.py"
def getArgs(net_path, trips_path, additional_path, iterations):
    args = [
        "-n", net_path,
        "-t", trips_path,
        "--additional", additional_path,
        "duarouter--additional-files", additional_path,
        "-l", str(iterations)
    ]
    return args

In [4]:
# Latitude e longitude de locais de interesse
coords = {
    "IC": (-22.813344, -47.063667),
    "FEEC": (-22.820788, -47.066489),
    "HOME": (-22.823781, -47.074111),
    "RU": (-22.817327, -47.071338),
    "IB": (-22.819330, -47.068836),
    "RS": (-22.815302, -47.062603),
    "BC": (-22.816370, -47.071218),
    "FEF": (-22.815240, -47.072680),
    "IFGW": (-22.817940, -47.067007),
    "IMECC": (-22.816385, -47.068329),
    "RA": (-22.822440, -47.065231),
    "IQ": (-22.817794, -47.068351),
    "FEQ": (-22.820118, -47.065556),
    "IG": (-22.813727, -47.069196)
}

In [5]:
radius = 150

In [6]:
def has_parking_spot(lanes, parkingAreas):
    # Example of parkingArea:
    # <parkingArea id="pa-1046248579#0" lane="-1046248579#0_0" roadsideCapacity="94" length="5.00"/>
    # Returns parkingArea id if there is a parking spot in the lane
    lane_ids = [lane.getID() for lane in lanes]
    for park in parkingAreas:
        if park.lane in lane_ids:
            return park.id
    return False

In [7]:
def getClosestEdges(lat, lon, radius, maxEdges=10):
    # Gets the 10 closest edges to the given lat, lon
    x, y = net.convertLonLat2XY(lon, lat)
    edges = net.getNeighboringEdges(x, y, radius)
    closestEdges = []
    if (len(edges) > 0):
        distanceAndEdges = sorted([(dist, edge) for edge, dist in edges], key=lambda x:x[0])

        ## Checking if the edge found can be used by passenger car
        for dist, edge in distanceAndEdges:
            if edge.allows('passenger'):
                closestEdges.append(edge)

    if len(edges) == 0:
        print(f'No edges found for {lat}, {lon}')
    return closestEdges

In [8]:
def getParkingSpot(lat, lon, radius, parkingAreas):
    # Get the parking spot closest to the given lat, lon
    # Used to set stops for the vehicles

    edges = getClosestEdges(lat, lon, radius)
    # Look for parking spots
    for i in range(len(edges)):
        parking_spot = has_parking_spot(edges[i].getLanes(), parkingAreas)
        if parking_spot:
            return parking_spot
    print("No parking spot found:", lat, lon)
    return None

In [9]:
def getPath(location_time_list, parkingAreas):
    # All that is needed to create the trip are the stops (parking areas) and the start and end edges.
    # The duarouter is responsible for finding the path between the edges going through the stops.
    # Here, we get the edges and stops that are going to be sent to LLAMA to create the trip.

    # 'coordinates' is a list of tuples with the latitude and longitude of the points of interest, for example IC, FEEC, IC means that
    # the vehicle will start from IC, stop at a parking lot close to FEEC, and then back to IC.
    # The first and last coordinates should be edges and the others should be parking spots.

    # Departure for 7 is 0, 8 is 100, 9 is 200 and so on
    stop_durations = []
    departures = list(location_time_list.keys())
    stop_durations.append(-1) # Indicates this is an edge and not a parking spot

    path = []
    locations = list(location_time_list.values())
    home = getClosestEdges(*locations[0], radius)[0].getID()
    path.append(home)
    
    for i in range(1, len(locations)-1):
        stop_durations.append(100 * (departures[i] - departures[i - 1]))
        path.append(getParkingSpot(*locations[i], radius, parkingAreas))
    
    path.append(home)
    stop_durations.append(-1)
    
    return path, stop_durations

In [10]:
def pathToXML(path, vehicleID, veh_type, departure_time, stop_durations):
    # Converts the path to the XML format that LLAMA understands
    xml = f'<trip id="{vehicleID}" type="{veh_type}" depart="{departure_time}" from="{path[0]}" to="{path[-1]}">\n'
    for i in range(1, len(path)-1):
        xml += f'\t<stop parkingArea="{path[i]}" duration="{stop_durations[i]}"/>\n'
    xml += '</trip>'
    return xml

Getting the routine for the students described

In [14]:
student_info = "A Computer Engineering student, a Computer Science student, two Biology students and one Physics student. One of the biology students loves sports and other biology does not have classes all day long. The Computer Engineering and Computer Science students have lunch and dinner at RU."
importlib.reload(LLAMAconnect)
response = LLAMAconnect.getResponse_trip(student_info)
json_response = json.loads(response)
print(response)

{
   "trip1": {
      "7": {"location": "HOME", "activity": "wake up"},
      "8": {"location": "FEEC", "activity": "classes"},
      "9": {"location": "FEEC", "activity": "classes"},
      "10": {"location": "FEEC", "activity": "classes"},
      "11": {"location": "FEEC", "activity": "classes"},
      "12": {"location": "RU", "activity": "lunch"},
      "13": {"location": "FEEC", "activity": "study"},
      "14": {"location": "FEEC", "activity": "classes"},
      "15": {"location": "FEEC", "activity": "classes"},
      "16": {"location": "FEEC", "activity": "classes"},
      "17": {"location": "FEEC", "activity": "classes"},
      "18": {"location": "RU", "activity": "dinner"},
      "19": {"location": "HOME", "activity": "relax"},
      "20": {"location": "HOME", "activity": "sleep"}
   },
   "trip2": {
      "7": {"location": "HOME", "activity": "wake up"},
      "8": {"location": "IC", "activity": "classes"},
      "9": {"location": "IC", "activity": "classes"},
      "10": {"locat

In [25]:
student_info = "10 students of any course. Create different trips for each one, try to define different activies for the students."
importlib.reload(LLAMAconnect)
response = LLAMAconnect.getResponse_trip(student_info)
json_response = json.loads(response)
print(response)

{
   "trip1": {
      "7": {"location": "HOME", "activity": "wake up"},
      "8": {"location": "IC", "activity": "study"},
      "9": {"location": "IC", "activity": "study"},
      "10": {"location": "RU", "activity": "breakfast"},
      "11": {"location": "IC", "activity": "study"},
      "12": {"location": "BC", "activity": "study"},
      "13": {"location": "BC", "activity": "study"},
      "14": {"location": "RU", "activity": "lunch"},
      "15": {"location": "BC", "activity": "study"},
      "16": {"location": "BC", "activity": "study"},
      "17": {"location": "IFGW", "activity": "study"},
      "18": {"location": "RS", "activity": "dinner"},
      "19": {"location": "HOME", "activity": "relax"},
      "20": {"location": "HOME", "activity": "sleep"}
   },
   "trip2": {
      "7": {"location": "HOME", "activity": "wake up"},
      "8": {"location": "IMECC", "activity": "study"},
      "9": {"location": "IMECC", "activity": "study"},
      "10": {"location": "HOME", "activity": 

Generating different houses for students based on a random number generator in the where students might live

In [26]:
def getHome():
    # top right -22.815539, -47.076272
    # top left: -22.818517, -47.079520
    # bottom left: -22.826410, -47.072591
    # bottom right: -22.823439, -47.068828
    X_MIN = 22815539
    X_MAX = 22826410
    Y_MIN = 47068828
    Y_MAX = 47079520
    while True:
        random_x = random.randint(X_MIN, X_MAX)
        random_y = random.randint(Y_MIN, Y_MAX)
        lat = - random_x / 10**6
        lon = - random_y / 10**6
        if (lat, lon) in coords.values():
            continue
        break
    return lat, lon

Getting the latitude and longitude for each of the locations

In [27]:
location_time_list = []
for j in range(len(json_response)):
    location_time_list.append({})
for i in range(1, len(json_response) + 1):

    trip = json_response[f'trip{i}']
    home = getHome() # Gets a random home location for each student
    last = home
    location_time_list[i - 1][7] = last
    
    for j in range(1, len(trip)):
        local = trip[f'{j + 7}']['location']

        if local == 'HOME':
            local_coords = home
        else:
            local_coords = coords[trip[f'{j + 7}']['location']]

        if local_coords != last:
            location_time_list[i - 1][j + 7] = local_coords
            last = local_coords

print(len(location_time_list))
pprint(location_time_list)

10
[{7: (-22.821568, -47.078269),
  8: (-22.813344, -47.063667),
  10: (-22.817327, -47.071338),
  11: (-22.813344, -47.063667),
  12: (-22.81637, -47.071218),
  14: (-22.817327, -47.071338),
  15: (-22.81637, -47.071218),
  17: (-22.81794, -47.067007),
  18: (-22.815302, -47.062603),
  19: (-22.821568, -47.078269)},
 {7: (-22.819176, -47.073753),
  8: (-22.816385, -47.068329),
  10: (-22.819176, -47.073753),
  11: (-22.816385, -47.068329),
  12: (-22.81637, -47.071218),
  14: (-22.817327, -47.071338),
  15: (-22.81637, -47.071218),
  17: (-22.816385, -47.068329),
  18: (-22.815302, -47.062603),
  19: (-22.819176, -47.073753)},
 {7: (-22.825431, -47.077079),
  8: (-22.817794, -47.068351),
  10: (-22.825431, -47.077079),
  11: (-22.817794, -47.068351),
  12: (-22.81637, -47.071218),
  14: (-22.817327, -47.071338),
  15: (-22.81637, -47.071218),
  17: (-22.817794, -47.068351),
  18: (-22.815302, -47.062603),
  19: (-22.825431, -47.077079)},
 {7: (-22.820567, -47.07564),
  8: (-22.813727,

Taking the locations and turning them into a XML trips, which are written to PATHGEN.trips.xml

In [32]:
def parseTripXML(location_time_list, parkingAreas, departure_times, styles, veh_types_per_student, n_vtypes=5):
    # Creates the XML for the trips
    # The number of vtypes is the number of vTypes created for each style
    # The current supported styles are "agg" for aggressive and "norm" for normal
    xml = '<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">\n'

    xml += '\n'
    xml += '<!-- Vehicles -->\n'
    vehdists = vehParameters.generateVehicleTypes(styles, n_vtypes)
    xml += vehParameters.parseVehiclesXML(vehdists, styles)

    xml += '\n'
    xml += '<!-- Trips -->\n'
    for i in range(len(location_time_list)):
        path, stop_durations = getPath(location_time_list[i], parkingAreas)
        xml += pathToXML(path, f'veh{i + 1}', veh_types_per_student[i], departure_times[i], stop_durations) + '\n'
    xml += '</routes>'
    with open('PATHGEN.trips.xml', 'w') as f:
        f.write(xml)
    return xml

In [33]:
departure_times = []
for i in range(len(location_time_list)):
    departure_times.append((list(location_time_list[i].keys())[0] - 7) * 20)

styles = ["agg", "norm"] # Currently only two styles, but can be expanded
n_vtypes = 5 
veh_style_per_student = []
for i in range(len(location_time_list)):
    if i < len(location_time_list) / 2:
        veh_style_per_student.append('agg')
    else: 
        veh_style_per_student.append('norm')   

xml = parseTripXML(location_time_list, parkingAreas, departure_times, styles, veh_style_per_student, n_vtypes)
print(xml)

<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">

<!-- Vehicles -->
<vTypeDistribution id="agg">
	<vType id="v_agg0" minGap="0.34" accel="2.63" decel="3.8" startupDelay="0.31" sigma="0.66" tau="0.71" maxSpeed="113.23" speedFactor="1.84" lcStrategic="6.68" lcCooperative="0.23" lcSpeedGain="7.53" lcKeepRight="3.81" lcOvertakeRight="0.77" lcSpeedGainLookahead="2.3" lcOvertakeDeltaSpeedFactor="0.65" lcPushy="0.87" lcAssertive="3.06" lcImpatience="0.75" lcTimeToImpatience="6.99" lcLaneDiscipline="2.15" lcSigma="0.77" lcAccelLat="1.59" probability="0.08558112381382266">
		<param key="device.rerouting.probability" value="1.0"/>
		<param key="device.rerouting.adaptation-steps" value="18"/>
		<param key="device.rerouting.adaptation-interval" value="10"/>
	</vType>
	<vType id="v_agg1" minGap="0.14" accel="3.38" decel="5.01" startupDelay="0.06" sigma="0.81" tau="0.73" maxSpeed="106.15" speedFactor="1.84" lcStrate

Getting alternative routes for every route found

In [30]:
# Run the duarouter to find alternative routes
# Currently this only works for less than 10 iterations
def getAltRoutes(net_path, trips_path, additional_path, iterations):
    for file in os.listdir('.'):
        if file.startswith('PATHGEN_') and file.endswith('.rou.alt.xml'):
            os.remove(file)
    
    try:
        subprocess.run(['python3', duaiterate_path] + getArgs(net_path, trips_path, additional_path, iterations), check=True)
        shutil.move(f'00{iterations-1}/PATHGEN_00{iterations-1}.rou.alt.xml', f'PATHGEN_00{iterations-1}.rou.alt.xml')
        for i in range(iterations):
            if os.path.exists(f'00{i}'):
                shutil.rmtree(f'00{i}')
        for file in os.listdir('.'):
            if file.startswith('PATHGEN_') and file.endswith('.rou.alt.xml'):
                os.rename(file, f'PATHGEN.rou.alt.xml')
    except subprocess.CalledProcessError as e:
        print(f"{e}")

In [34]:
getAltRoutes('osm.net.xml', 'PATHGEN.trips.xml', 'osm_stops.add.xml', 5)

> Executing step 0
>> Running router on PATHGEN.trips.xml
>>> Begin time: 2024-10-24 18:38:38.888761
>>> End time: 2024-10-24 18:38:39.480680
>>> Duration: 0:00:00.591919
<<
>> Running simulation
>>> Begin time: 2024-10-24 18:38:39.480751
>>> End time: 2024-10-24 18:38:40.947019
>>> Duration: 0:00:01.466268
<<
< Step 0 ended (duration: 0:00:02.058407)
------------------

> Executing step 1
>> Running router on 000/PATHGEN_000.rou.alt.xml
>>> Begin time: 2024-10-24 18:38:40.959758
>>> End time: 2024-10-24 18:38:41.531132
>>> Duration: 0:00:00.571374
<<
>> Running simulation
>>> Begin time: 2024-10-24 18:38:41.531241
>>> End time: 2024-10-24 18:38:43.156055
>>> Duration: 0:00:01.624814
<<
< Step 1 ended (duration: 0:00:02.196481)
------------------

> Executing step 2
>> Running router on 001/PATHGEN_001.rou.alt.xml
>>> Begin time: 2024-10-24 18:38:43.169572
>>> End time: 2024-10-24 18:38:43.754271
>>> Duration: 0:00:00.584699
<<
>> Running simulation
>>> Begin time: 2024-10-24 18:38:43.

To use this file, just set PATHGEN.rou.alt.xml as a trips file in osm.sumocfg